In [1]:
# Load the data
# Clean the data
# Feature Enginnering
# Preproccessing
# Modelling
# RandomSearching
# GridSearchings

In [2]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import (
    StandardScaler,
    RobustScaler,
    MinMaxScaler,
    MaxAbsScaler,
    OneHotEncoder,
    LabelEncoder,
    Normalizer,
)
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV,RidgeClassifier,RidgeClassifierCV
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier,AdaBoostClassifier,VotingClassifier,BaggingClassifier,RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier,XGBRFClassifier
from catboost import CatBoostClassifier,CatBoost
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
np.random.seed(42)

In [3]:
data = pd.read_csv('./data.csv')

In [4]:
def valid(model,X,y,val=False):
    preds = model.predict(X)
    if val is True:
        results = {
            'Val Precision':precision_score(y_true=y,y_pred=preds),
            'Val F1':f1_score(y_true=y,y_pred=preds),
            'Val Recall':recall_score(y_true=y,y_pred=preds),
            'Val Accuracy':accuracy_score(y_true=y,y_pred=preds),
        }
    else:
        results = {
            'Precision':precision_score(y_true=y,y_pred=preds),
            'F1':f1_score(y_true=y,y_pred=preds),
            'Recall':recall_score(y_true=y,y_pred=preds),
            'Accuracy':accuracy_score(y_true=y,y_pred=preds),
        }
    return results 
def fit(model,X_train,X_test,y_train,y_test,name):
    wandb.init(project=PROJECT_NAME,name=name)
    model = model
    model.fit(X_train,y_train)
    wandb.log(valid(model,X_train,y_train))
    wandb.log(valid(model,X_test,y_test,True))
    plot_roc_curve(model,X_train,y_train)
    plt.savefig('./img.png')
    wandb.log({'Roc':wandb.Image(cv2.imread('./img.png'))})
    plot_roc_curve(model,X_test,y_test)
    plt.savefig('./img.png')
    wandb.log({'Val Roc':wandb.Image(cv2.imread('./img.png'))})
    plot_confusion_matrix(model,X_train,y_train)
    plt.savefig('./img.png')
    wandb.log({'Confusion Matrix':wandb.Image(cv2.imread('./img.png'))})
    plot_confusion_matrix(model,X_test,y_test)
    plt.savefig('./img.png')
    wandb.log({'Val Confusion Matrix':wandb.Image(cv2.imread('./img.png'))})
    return model

In [7]:
from sklearn.compose import make_column_transformer

In [10]:
X = data.drop('gender',axis=1)
y = data['gender']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.125)

In [12]:
mct = make_column_transformer(
    (OneHotEncoder(handle_unknown='ignore'),['gender']),
    remainder='passthrough'
)
mct.fit(y_train)
y_train = mct.transform(y_train).toarray()
y_test = mct.transform(y_test).toarray()

IndexError: tuple index out of range